In [1]:
%pip install --quiet langchain langgraph tavily-python


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
user_input = "Write a report on the impact of climate change on polar bears."

In [3]:
from pydantic import BaseModel
from typing import List
import operator
from typing_extensions import Annotated

class QueryResult(BaseModel):
    title: str = None
    url: str = None
    resume: str = None
    

class ReportState(BaseModel):
    user_input: str = None
    final_response: str = None
    queries: List[str] = []
    queries_results: Annotated[List[QueryResult], operator.add]

In [4]:
import os
import dotenv

from langchain_openai import ChatOpenAI
from langgraph.graph import START, END, StateGraph
from langgraph.types import Send

dotenv.load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [5]:
llm = ChatOpenAI(model="gpt-4.1")

reasoning = {
    "effort": "medium",  # 'low', 'medium', or 'high'
    "summary": "auto",  # 'detailed', 'auto', or None
}

reasoning_llm = ChatOpenAI(
    model="o4-mini", reasoning=reasoning
)

In [6]:
agent_prompt = """
You are a research planner agent
You are  working on a project that aims to answer user's question
using resources found online

Your asnwer should be technical, detailed and well structured using up to date information
Cite facts, data and specific informations.

Here is the user input
<USER_INPUT>
{user_input}
</USER_INPUT>
"""

In [7]:
build_queries = agent_prompt + """
Your first objective is to build a list of queries that
will be used to find answers to the user's question.

Answer with anything between 3 and 5 queries.
"""

In [8]:
resume_search = agent_prompt + """
Your objective here is to analyze the web search results and make a synthesis of it.
Emphasize the most relevant information based on user's question.

After your work, another agent will use the synthesis to build a final response to the user,
so make sure the synthesis contains only useful information.

Be concise and clear, do not preamble.

Here is the web search results:
<SEARCH_RESULTS>
{search_results}
</SEARCH_RESULTS>
"""

In [9]:
build_final_response = agent_prompt + """
Your objective here is to develop a final response to the user using the reports made during
the web search, with their syntesis.

The response should contain something between 3 and 5 paragraphs.

Here is the web search results:
<SEARCH_RESULTS>
{search_results}
</SEARCH_RESULTS>

You must add reference citations with the number of the citation (e.g. [1], [2], etc.) at the end of each paragraph.
and the articles you used in each paragraph of your answer.
"""

In [10]:
def build_first_queries(state: ReportState):
    class QueryList(BaseModel):
        queries: List[str]
    
    user_input = state.user_input
    prompt = build_queries.format(user_input=user_input)
    query_llm = llm.with_structured_output(QueryList)

    result = query_llm.invoke(prompt)
    
    print("Generated queries from Query Builder: \n {result}\n", result)

    return{"queries": result.queries} #Formatted queries
    

In [11]:
def spawn_researchers(state: ReportState):
    return [Send("single_search", query) for query in state.queries]

In [12]:
from tavily import TavilyClient

def single_search(query: str):
    tavily_client = TavilyClient()
    results = tavily_client.search(query, 
                                   max_results=1,
                                   include_raw_content=False)
    
    url = results["results"][0]["url"]
    url_extraction = tavily_client.extract(url)

    if len(url_extraction["results"]) > 0:
        raw_content = url_extraction["results"][0]["raw_content"]
        prompt = resume_search.format(user_input=user_input, search_results=raw_content)
        llm_result = llm.invoke(prompt)

        query_results = QueryResult(title=results["results"][0]["title"],
                                   url=url,
                                   resume=llm_result.content)
        
        print("\n\nAgent for query:", query)
        print("Result:", query_results)
        
        return{"query_result": query_results}

In [13]:
def final_writer(state: ReportState):
    search_results = ""
    reference = ""
    for i, result in enumerate(state.queries_results):
        search_results += f"[{i+1}]\n\n"
        search_results += f"Title: {result.title}\n"
        search_results += f"URL: {result.url}\n"
        search_results += f"Content: {result.resume}\n\n"

        reference += f"[{i+1}] - {result.title} ({result.url})\n"
    
    prompt = build_final_response.format(user_input=state.user_input, search_results=search_results)
    llm_result = llm.invoke(prompt)

    final_response = llm_result + "\n\nReferences:\n" + reference

    return {"final_response": final_response}

In [14]:
builder = StateGraph(ReportState)
builder.add_node("build_first_queries", build_first_queries)
builder.add_node("single_search", single_search)
builder.add_node("final_writer", final_writer)

builder.add_edge(START, "build_first_queries")
builder.add_conditional_edges("build_first_queries", spawn_researchers, ["single_search"])
builder.add_edge("single_search", "final_writer")
builder.add_edge("final_writer", END)

graph = builder.compile()

In [15]:
graph.invoke({"user_input": user_input})

Generated queries from Query Builder: 
 {result}
 queries=['Recent studies on the effects of climate change on polar bear populations', 'How does sea ice loss affect polar bear hunting and survival?', 'Long-term trends in polar bear population numbers across the Arctic', 'Impact of climate change on polar bear reproduction and cub survival rates', 'Adaptation mechanisms of polar bears to changing Arctic environments']


Agent for query: Impact of climate change on polar bear reproduction and cub survival rates
-------------
Result: title="Climate change threatens bears' long-term survival - WWF Arctic" url='https://www.arcticwwf.org/the-circle/stories/climate-change-threatens-bears-long-term-survival/' resume='**Impact of Climate Change on Polar Bears – Synthesis**\n\n1. **Dependence on Sea Ice and Arctic Warming**\n   - Polar bears (Ursus maritimus) rely on sea ice as a platform for hunting seals, their primary food source.\n   - The Arctic is warming faster than any other region, res

{'user_input': 'Write a report on the impact of climate change on polar bears.',
 'final_response': ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[AIMessage(content='The impact of climate change on polar bears (Ursus maritimus) has become a critical focus of Arctic ecology and conservation. As the Arctic warms at approximately three times the global average rate, sea ice, which is fundamental to polar bears’ survival, is declining both in extent and duration. Loss of this sea ice habitat impairs the bears’ ability to hunt for their primary prey, seals, leading to reduced body condition, lower reproductive rates, and increased cub mortality. Scientific studies have documented a direct correlation between sea ice loss and declining polar bear populations across several subpopulations, notably in the Southern Beaufort Sea and Western Hudson Bay regions. Changes in the timing of sea ice melt and formation are reducing the hunting season, forcing bear